In [2]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.3MB 770kB/s ta 0:00:01
  Found existing installation: pip 8.1.2
    Uninstalling pip-8.1.2:
      Successfully uninstalled pip-8.1.2


In [3]:
!pip install psycopg2

  Using cached psycopg2-2.7.3-cp35-cp35m-manylinux1_x86_64.whl
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor
import pandas as pd
import numpy as np


In [16]:
connection = pg2.connect(host='postgres',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)

In [6]:
the_df = pd.read_pickle('../data/pages-df-text.p')

In [76]:
the_df.head()

,pageid,title,category,text
0,43385931,Data exploration,machine learning,data exploration is an approach similar to ini...
1,49082762,List of datasets for machine learning research,machine learning,these datasets are used for machine learning r...
2,233488,Machine learning,machine learning,machine learning is the subfield of computer s...
3,53587467,Outline of machine learning,machine learning,the following outline is provided as an overvi...
4,53198248,Singular statistical model,machine learning,in machine learning if a statistical model is ...


Dummy code for entering data into database  
``` 
cat_dict = {'machine learning':1, 'business software':2}
for k, v in cat_dict.items():
    insert v, k into category table

for idx, row in df.iterrows():
    insert row['pageid'], row['title'], row['text'] into page table
    insert row['pageid'], cat_dict[row['category']] into page_category table 
    ```

In [8]:
cat_dict = {'machine learning': 1, 'business software': 2} 

In [9]:
query = """
BEGIN;
INSERT INTO category 
(category_id, category_title)
VALUES (1, 'machine learning');
COMMIT;
"""

In [10]:
cursor.execute(query)

In [11]:
query = """
BEGIN;
INSERT INTO category 
(category_id, category_title)
VALUES (2, 'business software');
COMMIT;
"""

In [12]:
cursor.execute(query)

In [13]:
cursor.close()

In [14]:
import re
def cleaner(message):
    message = re.sub('\.+', ' ', message)
    message = re.sub('[^a-z0-9 ]','', message.lower())
    message = re.sub('\d+','NUMBER ',message)
    message = re.sub('\s+',' ',message)
    return message

In [67]:
connection = pg2.connect(host='postgres',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor) 

for i, row in the_df.iterrows():
    # check for duplicates
    select_query = """
    SELECT 1 
    FROM pages 
    WHERE page_id = {}
    """.format(row['pageid'])
    
    cursor.execute(select_query)
    if not cursor.fetchall():
        # clean title 
        title = row['title']
        title = cleaner(title)
    
        # query to input row into database
        row_query = """
        BEGIN;
        INSERT INTO pages
        (page_id, page_title, page_text)
        VALUES ({}, '{}', '{}');
        COMMIT;
        """.format(row['pageid'], title, row['text'], row['pageid'])
        
        cursor.execute(row_query) 
    
    # query to input row into page_category table 
    row_cat = row['category']
    cat_query = """
    BEGIN;
    INSERT INTO page_category
    (category_id, page_id)
    VALUES ({}, {});
    COMMIT;
    """.format(cat_dict[row_cat], row['pageid'])
    
    # input row into databases
    
    cursor.execute(cat_query)

In [77]:
connection = pg2.connect(host='postgres',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)

check_query = """

SELECT * FROM pages;

"""
cursor.execute(check_query)
test = pd.DataFrame(cursor.fetchall())

In [78]:
test.shape

(2654, 3)

In [79]:
cursor.close()

In [80]:
test.head()

,page_id,page_text,page_title
0,43385931,data exploration is an approach similar to ini...,data exploration
1,49082762,these datasets are used for machine learning r...,list of datasets for machine learning research
2,233488,machine learning is the subfield of computer s...,machine learning
3,53587467,the following outline is provided as an overvi...,outline of machine learning
4,53198248,in machine learning if a statistical model is ...,singular statistical model
